<a href="https://colab.research.google.com/github/alejandra2826/Alejandra-Trujillo/blob/main/PRACTICA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#**PRACTICA 3**

**Nombre:** Alejandra Elizabeth Trujillo Navarro
**e-mail:** alejandra.trujillo2826@alumnos.udg.mx

#**MODULES**


In [33]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import cauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

#**CLASSES**

In [34]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

#**FUNCTIONS**

In [35]:
######################################################################
# Brownian Motion Trajectory
######################################################################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    n_steps: number of steps the Brownian Trajectory will take -> int
    speed: speed of the trajectory or step size -> int
    s_pos: initial position -> [x,y] list
  Returns:
    BM_2d_df: DataFrame with x,y points of the full trajectory
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [36]:
# Correlated Random Walk
def crw(n_steps=1000, velocidad=5, s_pos=[0, 0], exponente_CRW=0.5):

    CRW_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    CRW_df = pd.concat([CRW_df, temp_df], ignore_index=True)

    # Inicializar el vector de velocidad
    velocidad_vector = np.array([velocidad, 0])

    for i in range(n_steps - 1):
        angulo_giro = np.random.standard_cauchy() * exponente_CRW
        matriz_rotacion = np.array([[np.cos(angulo_giro), -np.sin(angulo_giro)],
                                    [np.sin(angulo_giro), np.cos(angulo_giro)]])
        velocidad_vector = np.dot(matriz_rotacion, velocidad_vector)

        # Actualizar posición
        nueva_posicion = np.array([CRW_df.x_pos[i] + velocidad_vector[0], CRW_df.y_pos[i] + velocidad_vector[1]])
        temp_df = pd.DataFrame([{'x_pos': nueva_posicion[0], 'y_pos': nueva_posicion[1]}])
        CRW_df = pd.concat([CRW_df, temp_df], ignore_index=True)

    return CRW_df


In [37]:
def levy_flight_alternative(n_steps=100, init_speed=10, start_pos=[0, 0], crw_exponent=0.5, alpha=1.5, beta=0, loc=3.0):
    levy_df = pd.DataFrame(columns=['x_position', 'y_position'])
    temp_df = pd.DataFrame([{'x_position': start_pos[0], 'y_position': start_pos[1]}])
    levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        # Select turn angle and step size
        turn_angle = np.random.normal(scale=crw_exponent)
        step_size = np.random.standard_cauchy() * alpha + loc

        # Update position
        new_position = [levy_df.x_position[i] + np.cos(turn_angle) * step_size,
                        levy_df.y_position[i] + np.sin(turn_angle) * step_size]

        temp_df = pd.DataFrame([{'x_position': new_position[0], 'y_position': new_position[1]}])
        levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

    return levy_df

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [38]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('/content/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('/content/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('/content/brownian_6.csv')

In [39]:

def euclidean_distance_alternative(q, p):
    # Utilizar la función numpy para realizar los cálculos de manera más concisa
    distance = np.linalg.norm(np.array(q) - np.array(p))
    return distance

In [40]:
def path_length_alternative(trajectory):
    """
    Arguments:
        trajectory: Data set containing complete trajectory
    Returns:
        distance: calculated euclidean distance between 2 points
    """
    # Utilizar numpy para realizar cálculos de manera más eficiente
    differences = np.diff(trajectory, axis=0)
    distances = np.linalg.norm(differences, axis=1)

    # Calcular la distancia acumulativa
    cumulative_distances = np.cumsum(distances)

    # Crear un DataFrame con los resultados
    result_df = pd.DataFrame(cumulative_distances, columns=['PL'])
    return result_df

In [43]:
# MANDAR LLAMAR FUNCION
PH1 = path_length_alternative(BM_2d_df_3)
PH2 = path_length_alternative(BM_2d_df_6)
PH3 = path_length_alternative(CRW_2d_df_9)

In [44]:
# Plotting
# Init figure
fig_path_length = go.Figure()


# First trace BM1
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PH1.PL))+1,
    y = PH1.PL,
    name = 'path_length_BM_3',
    showlegend = True
))

# First trace BM6
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PH2.PL))+1,
    y = PH2.PL,
    name = 'path_length_BM_6',
    line = dict(width = 5),
    showlegend = True
))

# Third trace CRW
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PH3.PL))+1,
    y = PH3.PL,
    name = 'path_length_CRW_6',
    showlegend = True
))
fig_path_length.show()

**Actividad 2:** **Mean Squared Displacement - (BM vs CRW) (4 pts)**
* Generar una trayectoria tipo BM y una CRW.
* Implementar una función que calcule los valores de la curva de mean squared
displacement de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con plotly.





In [66]:
def calculate_msd(trajectory_data, window_size=50):
    """
    Calculate the Mean Squared Displacement (MSD) for a given trajectory over a specified window size.

    Parameters:
        trajectory_data (pd.DataFrame): DataFrame containing the trajectory points.
        window_size (int): The size of the window to calculate the MSD over.

    Returns:
        pd.DataFrame: A DataFrame containing the MSD values.
    """
    msd_values = []
    for i in range(window_size, len(trajectory_data)):
        squared_displacements = [
            distance.euclidean(trajectory_data.iloc[j-window_size], trajectory_data.iloc[j])**2
            for j in range(window_size, i+1)
        ]
        mean_squared_displacement = np.mean(squared_displacements)
        msd_values.append(mean_squared_displacement)

    return pd.DataFrame(msd_values, columns=['MSD'])

In [67]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('/content/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('/content/crw_6_9.csv')

In [69]:
MS1 = calculate_msd(BM_2d_df_6,500)
MS2 = calculate_msd(CRW_2d_df_9,500)

In [70]:
# Plotting
# Init figure
fig_path_length = go.Figure()


# First trace BM1
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(MS1.MSD))+1,
    y = MS1.MSD,
    name = 'path_length_BM_3',
    showlegend = True
))

# First trace BM6
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(MS2.MSD))+1,
    y = MS2.MSD,
    name = 'path_length_BM_6',
    line = dict(width = 5),
    showlegend = True
))

fig_path_length.show()

# **Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)**
* Generar dos CRWs con dos exponentes diferentes.
* Guardar trayectorias en pandas Data Frame.
* Implementar una función que calcule los valores de turning angle de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [81]:
#FUNCTION TURNING EAGLES
def calculate_turning_angles(trajectory):
    """
    Calculate turning angles from a trajectory.

    Parameters:
        trajectory (pd.DataFrame): DataFrame containing the trajectory with columns 'x' and 'y'.

    Returns:
        pd.DataFrame: DataFrame containing the turning angles.
    """
    # Utilizar numpy para realizar cálculos de manera más eficiente
    positions = np.array(trajectory)

    # Calcular las diferencias de posición
    differences = np.diff(positions, axis=0)

In [82]:
def calculate_turning_angles_alternative(trajectory):
    """
    Calculate turning angles from a trajectory using a different approach.

    Parameters:
        trajectory (pd.DataFrame): DataFrame containing the trajectory with columns 'x' and 'y'.

    Returns:
        pd.DataFrame: DataFrame containing the turning angles.
    """
    # Utilizar numpy para realizar cálculos de manera más eficiente
    positions = np.array(trajectory)

    # Calcular las diferencias de posición
    differences_prev = positions[:-2] - positions[1:-1]
    differences_next = positions[2:] - positions[1:-1]

    # Calcular los ángulos de giro utilizando arctan2
    angles_prev = np.arctan2(differences_prev[:, 1], differences_prev[:, 0])
    angles_next = np.arctan2(differences_next[:, 1], differences_next[:, 0])

    # Calcular las diferencias de ángulos
    turning_angles = (angles_next - angles_prev + np.pi) % (2 * np.pi) - np.pi

    # Crear un DataFrame con los resultados
    turning_angles_df = pd.DataFrame(turning_angles, columns=['Turning Angle'])
    return turning_angles_df

In [90]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('/content/crw_6_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('/content/crw_6_9.csv')

In [91]:
# calling the function
TURN1  = calculate_turning_angles_alternative(CRW_2d_df_6)
TURN2 = calculate_turning_angles_alternative(CRW_2d_df_9)